In [1]:
!pip install minio delta-spark==2.2.0

In [2]:
from minio import Minio
import io
import logging
import sys
import traceback
from datetime import datetime

import urllib3

from minio import Minio
from pyspark import SparkContext
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import unix_timestamp
import minio
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, when, avg, round

In [3]:
spark = (SparkSession.builder.appName("pyspark-rdd-demo-{}".format(datetime.today()))
        .master("spark://spark-master:7077")      
        .getOrCreate())

sqlContext = SQLContext(spark)
spark.sparkContext.getConf().getAll()

/usr/local/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark-3.3.2-bin-hadoop3/jars/delta-core_2.12-2.2.0.jar,file:///usr/local/spark-3.3.2-bin-hadoop3/jars/hadoop-aws-3.3.2.jar,file

In [4]:
df_orders = spark.read.csv("s3a://donnes-capteurs/donnes_capteurs_18-05-23.csv", header=True)


In [5]:
df_orders.limit(50).toPandas()

,timestamp,entrance_amount,exit_amount,temperature,humidity,parking_entrance,parking_exit,parking_actual_vehicle
0,2023-05-18 11:26:00.310649,31,42,23.98284652365134,51.51693541992328,1,2,35
1,2023-05-18 11:26:00.310649,18,0,18.209799550886544,49.07229505441757,4,4,128
2,2023-05-18 11:26:00.310649,20,9,20.139125780144397,34.6917614239392,3,4,237
3,2023-05-18 11:26:00.310649,-1,5,20.615199100988107,73.84951449694239,4,2,323
4,2023-05-18 11:26:00.310649,48,9,19.868111347969936,42.461563762012574,1,4,404
5,2023-05-18 11:26:00.310649,14,43,20.698440630556295,45.29607354356363,4,0,495
6,2023-05-18 11:26:00.310649,34,6,8.41287877716837,48.77352823073833,2,4,402
7,2023-05-18 11:26:00.310649,30,19,16.86485939410305,64.01811403528342,3,2,326
8,2023-05-18 11:26:00.310649,49,26,29.67589000399653,64.50520530798846,4,4,151
9,2023-05-18 11:26:00.310649,None,11,12.368722843604708,43.95874352407406,1,4,342


In [25]:
#traitement des données

# Transformer les valeurs nulles et négatives en 0 dans la colonne "entrance_amount"
df_orders = df_orders.withColumn("entrance_amount", when(col("entrance_amount").isNull() | (col("entrance_amount") < 0), 0).otherwise(col("entrance_amount")))

# Calculer la moyenne des valeurs dans la colonne "exit_amount"
exit_amount_avg = df_orders.select(avg(col("exit_amount")).cast("int")).first()[0]
exit_amount_avg = int(exit_amount_avg)

# Transformer les valeurs nulles ou négatives en moyenne des valeurs dans la colonne "exit_amount"
df_orders = df_orders.withColumn("exit_amount", when(col("exit_amount").isNull() | (col("exit_amount") < 0), exit_amount_avg).otherwise(col("exit_amount")))

# Transformer les valeurs nulles de la colonne parking_exit aux même valeurs de la colonne parking entrance
df_orders = df_orders.withColumn("parking_exit", when(col("parking_exit").isNull() | (col("parking_exit") == 0), col("parking_entrance")).otherwise(col("parking_exit")))

# Calculer la moyenne des valeurs dans la colonne "parking_actual_vehicle"
parking_actual_vehicle_avg = df_orders.select(avg(col("parking_actual_vehicle")).cast("int")).first()[0]
parking_actual_vehicle_avg = int(parking_actual_vehicle_avg)

# Transformer les valeurs négatives ou innexistantes en moyenne des valeurs dans la colonne "parking_actual_vehicle"
df_orders = df_orders.withColumn("parking_actual_vehicle", when(col("parking_actual_vehicle").isNull() | (col("parking_actual_vehicle") < 0), parking_actual_vehicle_avg).otherwise(col("parking_actual_vehicle")))

# Transformer les valeurs des colonnes de type string en int 
necessary_columns = ["entrance_amount", "exit_amount","parking_entrance", "parking_exit"]
for column in necessary_columns:
    df_orders = df_orders.withColumn(column, col(column).cast("int"))

# Transformer le type de la colonne parking_actual_vehicle en int
df_orders = df_orders.withColumn("parking_actual_vehicle", col("parking_actual_vehicle").cast("int"))
    
df_orders.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- entrance_amount: integer (nullable = true)
 |-- exit_amount: integer (nullable = true)
 |-- temperature: string (nullable = true)
 |-- humidity: string (nullable = true)
 |-- parking_entrance: integer (nullable = true)
 |-- parking_exit: integer (nullable = true)
 |-- parking_actual_vehicle: integer (nullable = true)



In [26]:
df_orders_final = df_orders.drop("timestamp", "temperature", "humidity")
df_orders_final.limit(50).toPandas()

,entrance_amount,exit_amount,parking_entrance,parking_exit,parking_actual_vehicle
0,31,42,1,2,35
1,18,23,4,4,128
2,20,9,3,4,237
3,0,5,4,2,323
4,48,9,1,4,404
5,14,43,4,4,495
6,34,6,2,4,402
7,30,19,3,2,326
8,49,26,4,4,151
9,0,11,1,4,342


In [27]:
spark.range(3000).write.format("delta").save("s3a://donnes-capteurs/testfile.txt", mode="overwrite")

In [28]:
sc = spark.sparkContext
fdd = sc.textFile("s3a://donnes-capteurs/testfile.txt")


In [73]:
def create_model(iter, features_cols, labelCol):
    # create LinearRegression estimator and set parameters
    lr = LinearRegression(maxIter=iter,
                          featuresCol=features_cols,
                          labelCol=labelCol)
    return lr


def set_train_and_validation_ds(data, seed):
    return data.randomSplit([0.7, 0.3], seed=seed)


def test_model(spark, model):
    # Methode pour tester le modèle avec un jeu de données test
    testData = spark.read.format("libsvm").load("s3a://donnes-capteurs/testfile.txt")
    predictions = model.transform(testData)
    predictions.show()


def save_model(model):
    # Methode pour sauvegarder le modèle vers un lieu spécifié
    #model.save("saved_model/model_test")
    model.write().overwrite().save("saved_model/model_test")

In [30]:
# Création de l'objet featureassembler
featureassembler = VectorAssembler(inputCols=necessary_columns, outputCol='necessary_columns')

In [31]:
# Transforme les colonnes necessaires en un vecteur
output = featureassembler.transform(df_orders_final)
output.show()

+---------------+-----------+----------------+------------+----------------------+-------------------+
|entrance_amount|exit_amount|parking_entrance|parking_exit|parking_actual_vehicle|  necessary_columns|
+---------------+-----------+----------------+------------+----------------------+-------------------+
|             31|         42|               1|           2|                    35|[31.0,42.0,1.0,2.0]|
|             18|         23|               4|           4|                   128|[18.0,23.0,4.0,4.0]|
|             20|          9|               3|           4|                   237| [20.0,9.0,3.0,4.0]|
|              0|          5|               4|           2|                   323|  [0.0,5.0,4.0,2.0]|
|             48|          9|               1|           4|                   404| [48.0,9.0,1.0,4.0]|
|             14|         43|               4|           4|                   495|[14.0,43.0,4.0,4.0]|
|             34|          6|               2|           4|              

In [32]:
# Extraction
final_data = output.select("necessary_columns", "parking_actual_vehicle")
final_data.show()

+-------------------+----------------------+
|  necessary_columns|parking_actual_vehicle|
+-------------------+----------------------+
|[31.0,42.0,1.0,2.0]|                    35|
|[18.0,23.0,4.0,4.0]|                   128|
| [20.0,9.0,3.0,4.0]|                   237|
|  [0.0,5.0,4.0,2.0]|                   323|
| [48.0,9.0,1.0,4.0]|                   404|
|[14.0,43.0,4.0,4.0]|                   495|
| [34.0,6.0,2.0,4.0]|                   402|
|[30.0,19.0,3.0,2.0]|                   326|
|[49.0,26.0,4.0,4.0]|                   151|
| [0.0,11.0,1.0,4.0]|                   342|
|[25.0,42.0,4.0,1.0]|                   467|
| [4.0,34.0,1.0,1.0]|                    68|
| [30.0,1.0,1.0,1.0]|                   367|
|[38.0,29.0,2.0,2.0]|                   361|
|[46.0,30.0,4.0,2.0]|                   347|
|[43.0,40.0,4.0,4.0]|                   302|
| [40.0,4.0,1.0,1.0]|                   347|
| [28.0,1.0,1.0,2.0]|                   440|
|[49.0,20.0,4.0,2.0]|                   394|
|[15.0,36.

In [33]:
# Initialisation du model de  Regression linéaire
model = LinearRegression(featuresCol='necessary_columns', labelCol='parking_actual_vehicle')

In [34]:
# Train Test Split
train_data, test_data = final_data.randomSplit(weights=[0.7,0.3], seed=42)

In [35]:
# Entrainement du model
model = model.fit(train_data)

In [36]:
# L'intercept représente la valeur de la variable cible lorsque toutes les caractéristiques sont nulles
model.intercept

253.89334033962893

In [55]:
# Evalutation du model sur train_data
pred_train = model.evaluate(train_data)
res = pred_train.predictions
res = res.withColumn("prediction",col("prediction").cast("integer"))
res.show()

+-----------------+----------------------+----------+
|necessary_columns|parking_actual_vehicle|prediction|
+-----------------+----------------------+----------+
|[0.0,1.0,1.0,2.0]|                   179|       249|
|[0.0,1.0,2.0,1.0]|                   215|       249|
|[0.0,1.0,2.0,2.0]|                   444|       247|
|[0.0,1.0,2.0,4.0]|                   343|       244|
|[0.0,1.0,3.0,2.0]|                   199|       246|
|[0.0,1.0,3.0,3.0]|                    93|       245|
|[0.0,1.0,4.0,4.0]|                    80|       242|
|[0.0,1.0,4.0,4.0]|                    95|       242|
|[0.0,1.0,4.0,4.0]|                   396|       242|
|[0.0,2.0,1.0,4.0]|                   384|       245|
|[0.0,2.0,2.0,1.0]|                    79|       249|
|[0.0,2.0,2.0,2.0]|                   202|       247|
|[0.0,2.0,2.0,4.0]|                   278|       244|
|[0.0,2.0,3.0,3.0]|                    62|       244|
|[0.0,2.0,3.0,3.0]|                   323|       244|
|[0.0,2.0,3.0,3.0]|         

In [57]:
pred = model.evaluate(test_data)
res = pred.predictions
res = res.withColumn("prediction",col("prediction").cast("integer"))
res.show()

+-----------------+----------------------+----------+
|necessary_columns|parking_actual_vehicle|prediction|
+-----------------+----------------------+----------+
|[0.0,1.0,2.0,2.0]|                   147|       247|
|[0.0,1.0,3.0,2.0]|                   300|       246|
|[0.0,1.0,3.0,3.0]|                   100|       245|
|[0.0,1.0,4.0,2.0]|                   134|       245|
|[0.0,2.0,1.0,2.0]|                   248|       249|
|[0.0,2.0,1.0,3.0]|                     6|       247|
|[0.0,2.0,1.0,3.0]|                   339|       247|
|[0.0,2.0,2.0,3.0]|                   372|       246|
|[0.0,2.0,2.0,4.0]|                   436|       244|
|[0.0,2.0,3.0,3.0]|                   109|       244|
|[0.0,2.0,3.0,3.0]|                   161|       244|
|[0.0,2.0,4.0,2.0]|                   333|       245|
|[0.0,2.0,4.0,4.0]|                    13|       242|
|[0.0,2.0,4.0,4.0]|                   156|       242|
|[0.0,2.0,4.0,4.0]|                   451|       242|
|[0.0,3.0,1.0,3.0]|         

In [74]:
# Sauvegarde du model ml
save_model(model)

Py4JJavaError: An error occurred while calling o1180.save.
: org.apache.spark.SparkException: Could not load requested format parquet for org.apache.spark.ml.regression.LinearRegressionModel (parquet+org.apache.spark.ml.regression.LinearRegressionModel) had List(org.apache.spark.ml.regression.InternalLinearRegressionModelWriter@23affcd9, org.apache.spark.ml.regression.PMMLLinearRegressionModelWriter@34344106, org.apache.spark.ml.clustering.InternalKMeansModelWriter@176486c6, org.apache.spark.ml.clustering.PMMLKMeansModelWriter@2649a2a7)supporting List(internal+org.apache.spark.ml.regression.LinearRegressionModel, pmml+org.apache.spark.ml.regression.LinearRegressionModel, internal+org.apache.spark.ml.clustering.KMeansModel, pmml+org.apache.spark.ml.clustering.KMeansModel)
	at org.apache.spark.ml.util.GeneralMLWriter.saveImpl(ReadWrite.scala:249)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:168)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Meth